### Open Source LLM

We can combine LLamaCPP or [Ollama integrated](https://ollama.ai) to use [Code LLaMA](https://about.fb.com/news/2023/08/code-llama-ai-for-coding/) /blog/run-code-llama-locally）.

Note：Please update `llama-cpp-python` , so that the `gguf` [format](https://github.com/abetlen/llama-cpp-python/pull/633) can be used here.

````
CMAKE_ARGS =“-DLLAMA_METAL = on”FORCE_CMAKE = 1 /Users/rlm/miniforge3/envs/llama2/bin/pip install -U llama-cpp-python --no-cache-dir
````

Check the lastest code-llama model [here](https://huggingface.co/TheBloke/CodeLlama-13B-Instruct-GGUF/tree/main).

In [5]:
!pip install -U llama-cpp-python
!pip install openai tiktoken chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 90.4 MB/s eta 0:00:00ta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 13.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.28-cp38-cp38-manylinux_2_31_x86_64.whl size=2113205 sha256=b4ce51d3f00900add3524f1b5ce69bc7602159f986c6ca4e7bc86ca7090370cb
  Stored in directory: /srv/users/rudxia/.cache/pip/wheels/12/12/f8/9e08a6fefe1455c9f86713e5c834fa8cfc117c64ca82c53c25
Successfully built llama-cpp-python


In [1]:
from langchain import PromptTemplate
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path="/srv/users/rudxia/Developer_MICCAI/CodeLlama/model/codellama-13b-instruct.Q3_K_M.gguf",
    n_ctx=100000,
    n_gpu_layers=1,
    n_batch=512,
    f16_kv=False,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 363 tensors from /srv/users/rudxia/Developer_MICCAI/CodeLlama/model/codellama-13b-instruct.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = codellama_codellama-13b-instruct-hf
llama_model_loader: - kv   2:                       llama.context_length u32              = 16384
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_

## **Prompt after formatting**

In [38]:
question1="Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Enhance the program to display timestamps along with the sorted user inputs."
prompt=PromptTemplate.from_template(question1)

default_chain=LLMChain(llm=llm, prompt=prompt, verbose=True)
default_chain.run(prompt=question1)



> Entering new LLMChain chain...
Prompt after formatting:
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Enhance the program to display timestamps along with the sorted user inputs.


#include <iostream>
#include <iomanip>
using namespace std;
int main() {
	cout<<"Please enter the number:";
	double num, temp = 0;
	cin>>num;
	if (num % 2 == 0) {
		temp = num / 2;
	} else if ((num + 1) % 2 == 0)) {
		temp = ((num + 1))) / 2;
	}
	cout<<"The number is: " << temp << endl;





return 0;
















> Finished chain.



llama_print_timings:        load time =    2303.34 ms
llama_print_timings:      sample time =      41.32 ms /   154 runs   (    0.27 ms per token,  3727.01 tokens per second)
llama_print_timings: prompt eval time =    2301.16 ms /    64 tokens (   35.96 ms per token,    27.81 tokens per second)
llama_print_timings:        eval time =   21380.00 ms /   153 runs   (  139.74 ms per token,     7.16 tokens per second)
llama_print_timings:       total time =   24370.36 ms


'\n\n#include <iostream>\n#include <iomanip>\nusing namespace std;\nint main() {\n\tcout<<"Please enter the number:";\n\tdouble num, temp = 0;\n\tcin>>num;\n\tif (num % 2 == 0) {\n\t\ttemp = num / 2;\n\t} else if ((num + 1) % 2 == 0)) {\n\t\ttemp = ((num + 1))) / 2;\n\t}\n\tcout<<"The number is: " << temp << endl;\n\n\n\n\n\nreturn 0;\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

In [7]:
question2="Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. \
Ensure the program sorts the user inputs in ascending order based on the provided numeric input. \
Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. \
Use a suitable data structure (e.g., std::vector or std::list) to efficiently store and manage the user input records. \
Implement a function to display the sorted user input records with timestamps. \
Enhance the program to display timestamps along with the sorted user inputs. \
Use the C++ standard library (e.g., std::chrono::system_clock) for handling timestamps. \
Ensure that the timestamp is updated each time a new user input is added. \
Provide proper formatting for the timestamp in the output."

prompt=PromptTemplate.from_template(question2)

default_chain=LLMChain(llm=llm, prompt=prompt, verbose=True)
default_chain.run(prompt=question2)



> Entering new LLMChain chain...
Prompt after formatting:
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. Use a suitable data structure (e.g., std::vector or std::list) to efficiently store and manage the user input records. Implement a function to display the sorted user input records with timestamps. Enhance the program to display timestamps along with the sorted user inputs. Use the C++ standard library (e.g., std::chrono::system_clock) for handling timestamps. Ensure that the timestamp is updated each time a new user input is added. Provide proper formatting for the timestamp in the output.


Llama.generate: prefix-match hit









> Finished chain.



llama_print_timings:        load time =    5911.12 ms
llama_print_timings:      sample time =       2.00 ms /     7 runs   (    0.29 ms per token,  3498.25 tokens per second)
llama_print_timings: prompt eval time =    1308.46 ms /    38 tokens (   34.43 ms per token,    29.04 tokens per second)
llama_print_timings:        eval time =     644.31 ms /     6 runs   (  107.38 ms per token,     9.31 tokens per second)
llama_print_timings:       total time =    1980.02 ms /    44 tokens


'\n\n\n\n\n\n'

## **Prompt directly**

In [11]:
prompt1 = """
Question: Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Enhance the program to display timestamps along with the sorted user inputs.
"""
llm(prompt1)

\

Llama.generate: prefix-match hit


begin{code}
#include <iostream>
using namespace std;
int main() {
    cout << "Enter an integer: ";
    int userInput = 0, tempIntVar = 0;
    cin >> userInput;
    if (userInput > 1) {
        cout << "\n";
        for (int counter = 2; counter <= userInput; ++counter) {
            cout << counter << " ";
            if (counter % 3 == 0)) {
                cout << endl;
            } else if ((counter >= 5) && (counter % 5 == 0))))))) {
                        cout << "\n";
                        for (int counter = 2; counter <= userInput; ++counter) {
                                    cout << counter << " ";
                                    if (counter % 3 == 0)) {
                                                            cout << endl;
                                                        } else if (((counter >= 5) && (counter % 5 == 0))) {
                                                                                                                cout << "\n";
       


llama_print_timings:        load time =    5911.12 ms
llama_print_timings:      sample time =      69.68 ms /   256 runs   (    0.27 ms per token,  3674.04 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   37557.97 ms /   256 runs   (  146.71 ms per token,     6.82 tokens per second)
llama_print_timings:       total time =   38747.29 ms /   257 tokens


'\\begin{code}\n#include <iostream>\nusing namespace std;\nint main() {\n    cout << "Enter an integer: ";\n    int userInput = 0, tempIntVar = 0;\n    cin >> userInput;\n    if (userInput > 1) {\n        cout << "\\n";\n        for (int counter = 2; counter <= userInput; ++counter) {\n            cout << counter << " ";\n            if (counter % 3 == 0)) {\n                cout << endl;\n            } else if ((counter >= 5) && (counter % 5 == 0))))))) {\n                        cout << "\\n";\n                        for (int counter = 2; counter <= userInput; ++counter) {\n                                    cout << counter << " ";\n                                    if (counter % 3 == 0)) {\n                                                            cout << endl;\n                                                        } else if (((counter >= 5) && (counter % 5 == 0))) {\n                                                                                                            

In [12]:
prompt2 = """
Question: Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. \
Ensure the program sorts the user inputs in ascending order based on the provided numeric input. \
Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. \
Use a suitable data structure (e.g., std::vector or std::list) to efficiently store and manage the user input records. \
Implement a function to display the sorted user input records with timestamps. \
Enhance the program to display timestamps along with the sorted user inputs. \
Use the C++ standard library (e.g., std::chrono::system_clock) for handling timestamps. \
Ensure that the timestamp is updated each time a new user input is added. \
Provide proper formatting for the timestamp in the output.
"""
llm(prompt2)

Llama.generate: prefix-match hit


\begin{code}
#include <iostream>
using namespace std;
void display_sorted(const int &input)) {
    return 0;
}
\end{code}
Comment: What have you tried so far? Please take some time to read [the help pages](http://stackoverflow.com/help), especially the sections named ["What topics can I ask about here?"](http://stackoverflow.com/help/on-topic) and ["How do I ask a good question?"](http://stackoverflow.com/help/how-to-ask).
Comment: Welcome to Stack Overflow. Please read https://stackoverflow.com/questions/409019/c-sorting-algorithm-program-designed-by-me, and then [edit] your question to include the source code that you are having trouble with, and ask a specific question about that code.
Comment: What is the purpose of the `display_sorted` function?  What is its expected behavior?  Please read http://www.catb.org/esr/faqs/smart-questions.html then [edit] your question to include concise information about what you are asking,


llama_print_timings:        load time =    5911.12 ms
llama_print_timings:      sample time =      77.97 ms /   256 runs   (    0.30 ms per token,  3283.23 tokens per second)
llama_print_timings: prompt eval time =    4977.26 ms /   138 tokens (   36.07 ms per token,    27.73 tokens per second)
llama_print_timings:        eval time =   39171.19 ms /   255 runs   (  153.61 ms per token,     6.51 tokens per second)
llama_print_timings:       total time =   45328.29 ms /   393 tokens


'\\begin{code}\n#include <iostream>\nusing namespace std;\nvoid display_sorted(const int &input)) {\n    return 0;\n}\n\\end{code}\nComment: What have you tried so far? Please take some time to read [the help pages](http://stackoverflow.com/help), especially the sections named ["What topics can I ask about here?"](http://stackoverflow.com/help/on-topic) and ["How do I ask a good question?"](http://stackoverflow.com/help/how-to-ask).\nComment: Welcome to Stack Overflow. Please read https://stackoverflow.com/questions/409019/c-sorting-algorithm-program-designed-by-me, and then [edit] your question to include the source code that you are having trouble with, and ask a specific question about that code.\nComment: What is the purpose of the `display_sorted` function?  What is its expected behavior?  Please read http://www.catb.org/esr/faqs/smart-questions.html then [edit] your question to include concise information about what you are asking,'